In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from tqdm import tqdm
import ipywidgets as widgets
import time
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.webdriver.support.ui import Select

In [2]:
import pandas as pd
import numpy as np

In [4]:
options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("/home/lim/dev_ws/eda/driver/chromedriver"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

In [9]:
year = driver.find_element(By.ID, "sSearchYearFrom")
years = []
years_list = year.find_elements(By.TAG_NAME, "option")
for yearly in years_list:
    yearly = yearly.get_attribute("value")
    years.append(yearly)
years = years[9:]
years

['2013',
 '2014',
 '2015',
 '2016',
 '2017',
 '2018',
 '2019',
 '2020',
 '2021',
 '2022',
 '2023',
 '2024']

In [10]:
year.send_keys(years[-1])

In [11]:
movie_names = []
page = driver.page_source
soup = BeautifulSoup(page, "html.parser")
movies = soup.find("table", "tbl_comm th_sort")
moviesList = movies.find_all("span", class_="ellip per90")
for movie in moviesList:
    movie = movie.find("a").text
    movie_names.append(movie)

movie_names

['파묘',
 '범죄도시4',
 '인사이드 아웃 2',
 '베테랑2',
 '파일럿',
 '웡카',
 '모아나 2',
 '소방관',
 '탈주',
 '듄: 파트2',
 '에이리언: 로물루스',
 '데드풀과 울버린',
 '위키드',
 '핸섬가이즈',
 '하이재킹',
 '쿵푸팬더4',
 '베놈: 라스트 댄스',
 '시민덕희',
 '퓨리오사: 매드맥스 사가',
 '슈퍼배드 4',
 '하얼빈',
 '외계+인 2부',
 '위시',
 '서울의 봄',
 '사랑의 하츄핑',
 '그녀가 죽었다',
 '건국전쟁',
 '노량: 죽음의 바다',
 '히든페이스',
 '댓글부대',
 '글래디에이터 Ⅱ',
 '혹성탈출: 새로운 시대',
 '대도시의 사랑법',
 '청설',
 '명탐정 코난: 100만 달러의 펜타그램',
 '극장판 하이큐!! 쓰레기장의 결전',
 '행복의 나라',
 '탈출: 프로젝트 사일런스',
 '와일드 로봇',
 '보통의 가족',
 '원더랜드',
 '트위스터스',
 '조커: 폴리 아 되',
 '아마존 활명수',
 '콰이어트 플레이스: 첫째 날',
 '설계자',
 '고질라 X 콩: 뉴 엠파이어',
 '빅토리',
 '귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로',
 '무파사: 라이온 킹']

In [12]:
movie_viewers = []
movieL = driver.find_elements(By.ID, "td_audiAcc")
for movie in movieL:
    movie_viewers.append(movie.text)

movie_viewers

['11,914,716',
 '11,502,779',
 '8,799,013',
 '7,525,339',
 '4,718,036',
 '3,531,717',
 '3,238,576',
 '2,983,233',
 '2,561,872',
 '2,019,129',
 '2,002,033',
 '1,977,010',
 '1,971,872',
 '1,776,995',
 '1,774,150',
 '1,771,738',
 '1,763,723',
 '1,715,064',
 '1,606,378',
 '1,562,001',
 '1,481,814',
 '1,430,121',
 '1,403,486',
 '1,273,640',
 '1,239,064',
 '1,237,103',
 '1,173,927',
 '1,135,516',
 '1,008,846',
 '972,991',
 '909,478',
 '907,155',
 '879,096',
 '801,345',
 '753,606',
 '744,521',
 '710,620',
 '689,603',
 '682,036',
 '650,336',
 '625,908',
 '620,240',
 '612,831',
 '606,612',
 '552,619',
 '525,369',
 '510,370',
 '508,467',
 '497,320',
 '494,680']

In [13]:
movie_genre = []
for movie in tqdm(movie_names):
    print(movie)
    link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie)))
    link.click()

    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")

    for dt in soup.find_all("dt", text="요약정보"):
        dd = dt.find_next("dd")  # Get the associated <dd> element
        if dd:
            # Extract and clean the genre information
            raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
            parts = raw_text.split("|")  # Split by '|'
            genre_parts = [part.strip() for part in parts if "," in part]  # Filter parts with commas (genres)
            for genre in genre_parts:
                # Further split by commas to separate multiple genres
                movie_genre.extend([g.strip() for g in genre.split(",")])
                genres = [
                    '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', 
                    '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스'
                ]

                movie_genre = [item for item in movie_genre if item in genres]

    close = driver.find_element(By.CLASS_NAME, "close")
    close.click()

movie_genre, len(movie_genre)

  0%|          | 0/50 [00:00<?, ?it/s]

파묘


/tmp/ipykernel_6880/1525041506.py:10: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for dt in soup.find_all("dt", text="요약정보"):
  2%|▏         | 1/50 [00:03<02:30,  3.06s/it]

범죄도시4


  4%|▍         | 2/50 [00:05<02:03,  2.58s/it]

...

귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로


 98%|█████████▊| 49/50 [01:35<00:01,  1.48s/it]

무파사: 라이온 킹


100%|██████████| 50/50 [01:36<00:00,  1.93s/it]


(['미스터리',
  '공포(호러)',
  '액션',
  '범죄',
  '액션',
  '범죄',
  '판타지',
  '드라마',
  '공포(호러)',
  'SF',
  '액션',
  '코미디',
  'SF',
  '판타지',
  '뮤지컬',
  '범죄',
  '액션',
  '애니메이션',
  '액션',
  '코미디',
  '액션',
  'SF',
  '스릴러',
  '애니메이션',
  '액션',
  '어드벤처',
  '코미디',
  '미스터리',
  '스릴러',
  '사극',
  '액션',
  '범죄',
  '드라마',
  '액션',
  '사극',
  '액션',
  'SF',
  '드라마',
  '멜로/로맨스',
  '액션',
  '어드벤처',
  '드라마',
  '범죄',
  '드라마',
  '뮤지컬',
  '범죄',
  '드라마',
  '액션',
  '어드벤처',
  'SF'],
 50)

In [80]:
movie_names = []
movie_viewers = []
movie_genre = []
movie_years = []
movieG = []

options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("C:/Users/AAA/.vscode/workspace/AddInEdu/driver/chromedriver-win64/chromedriver.exe"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
select = Select(year_dropdown)  # Use Select to interact with the dropdown
years_list = select.options  # Get all options in the dropdown

years = []
for option in years_list:
    years.append(option.get_attribute("value"))

# Limit to the last few years (example: from the 10th year onward)
years = years[9:]
years = years[:2]
# Loop through the years and perform scraping
for yearly in tqdm(years):
    year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
    select = Select(year_dropdown)  # Re-initialize the Select object
    
    driver.execute_script("window.scrollTo(0, 0)")
    select.select_by_value(yearly)  # Select the year from the dropdown
    
    # Click the "show" button to load the data
    show = driver.find_element(By.CLASS_NAME, "wrap_btn")
    show.click()
    time.sleep(1)

    movieN = []
    # Wait for the page to load and parse it
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    movies = soup.find("table", "tbl_comm th_sort")
    moviesList = movies.find_all("span", class_="ellip per90")
    for movie in moviesList:
        movie = movie.find("a").text
        movie_names.append(movie)
        movieN.append(movie)
        movie_years.append(yearly)
    
    movieL = driver.find_elements(By.ID, "td_audiAcc")
    for movie in movieL:
        movie_viewers.append(movie.text)

    for movie in tqdm(movieN):
        movieG = []
        link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie)))
        link.click()

        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

        dt = soup.find("dt", text="요약정보")
        if dt:
            dd = dt.find_next("dd")  # Get the associated <dd> element
            if dd:
                # Extract and clean the genre information
                raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
                parts = raw_text.split("|")  # Split by '|'
                genre_parts = [part.strip() for part in parts if "," in part]  # Filter parts with commas (genres)
                for genre in genre_parts:
                    # Further split by commas to separate multiple genres
                    movieG.extend([g.strip() for g in genre.split(",")])
                    # Filter only recognized genres
                    genres = [                print(parts)
                        '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', 
                        '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스'
                    ]
                    current_movie_genres = [item for item in movieG if item in genres]
                    print(current_movie_genres)
                movie_genre.append(current_movie_genres)
            else:
                # If no <dd> found, append an empty list
                movie_genre.append([])
        else:
            # If no "요약정보" section, append an empty list
            movie_genre.append([])

        close = driver.find_element(By.CLASS_NAME, "close")
        close.click()

len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

  0%|          | 0/2 [00:00<?, ?it/s]C:\Users\AAA\AppData\Local\Temp\ipykernel_18688\2590976463.py:66: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  dt = soup.find("dt", text="요약정보")


['장편 ', ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t                       \t\n\t                       \t\t\t코미디,\n\t                       \t\n\t                       \t\t\t드라마\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t127분 0초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                           \t\t\t15세이상관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t                        \t한국\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t', '\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\

['장편 ', ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t                       \t\n\t                       \t\t\tSF,\n\t                       \t\n\t                       \t\t\t액션\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t125분 46초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                           \t\t\t15세이상관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t                        \t한국,\n\t                        \n\t                        \t체코\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t', '']
['SF', '액션']
['SF', '액션']


['장편 ', ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t                       \t\n\t                       \t\t\t사극,\n\t                       \t\n\t                       \t\t\t드라마\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t139분 40초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                           \t\t\t15세이상관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t                        \t한국\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t', '\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\

['장편 ', ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t                       \t\n\t                       \t\t\t액션,\n\t                       \t\n\t                       \t\t\t어드벤처,\n\t                       \t\n\t                       \t\t\tSF\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t130분 22초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\t\t\n                           \t\t\t12세이상관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t                        \t미국\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t', '']
['액션', '어드벤처', 'SF']


['장편 ', ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t                       \t\n\t                       \t\t\t액션\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t120분 0초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                           \t\t\t15세이상관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t                        \t한국,\n\t                        \n\t                        \t독일\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t', '\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\

['장편 ', ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t                       \t\n\t                       \t\t\t액션,\n\t                       \t\n\t                       \t\t\t코미디,\n\t                       \t\n\t                       \t\t\t드라마\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t123분 47초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                           \t\t\t15세이상관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t', '\n\t\t\t\t\t\t\t\n\t                        \t한국\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t', '\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\

  0%|          | 0/2 [00:12<?, ?it/s]


KeyboardInterrupt: 

In [75]:
len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

(100, 100, 100, 100)

In [16]:
movie_genre, len(movie_genre)

(['미스터리',
  '공포(호러)',
  '액션',
  '범죄',
  '액션',
  '범죄',
  '판타지',
  '드라마',
  '공포(호러)',
  'SF',
  '액션',
  '코미디',
  'SF',
  '판타지',
  '뮤지컬',
  '범죄',
  '액션',
  '애니메이션',
  '액션',
  '코미디',
  '액션',
  'SF',
  '스릴러',
  '애니메이션',
  '액션',
  '어드벤처',
  '코미디',
  '미스터리',
  '스릴러',
  '사극',
  '액션',
  '범죄',
  '드라마',
  '액션',
  '사극',
  '액션',
  'SF',
  '드라마',
  '멜로/로맨스',
  '액션',
  '어드벤처',
  '드라마',
  '범죄',
  '드라마',
  '뮤지컬',
  '범죄',
  '드라마',
  '액션',
  '어드벤처',
  'SF'],
 50)

In [46]:
options = webdriver.ChromeOptions()

prefs = {"download.default_directory" : "/home/lim/dev_ws/eda/data", "download.prompt_for_download":False}
options.add_experimental_option("prefs", prefs)

url = "https://www.kobis.or.kr/kobis/business/stat/boxs/findYearlyBoxOfficeList.do"
driver = webdriver.Chrome(service=Service("C:/Users/AAA/.vscode/workspace/AddInEdu/driver/chromedriver-win64/chromedriver.exe"), options=options)

driver.get(url)
driver.set_window_size(1800, 1300)

year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
select = Select(year_dropdown)  # Use Select to interact with the dropdown
years_list = select.options  # Get all options in the dropdown

years = []
for option in years_list:
    years.append(option.get_attribute("value"))

# Limit to the last few years (example: from the 10th year onward)
years = years[9:]
years = years[6:]
# Loop through the years and perform scraping
for yearly in tqdm(years):
    year_dropdown = driver.find_element(By.ID, "sSearchYearFrom")
    select = Select(year_dropdown)  # Re-initialize the Select object
    
    driver.execute_script("window.scrollTo(0, 0)")
    select.select_by_value(yearly)  # Select the year from the dropdown
    print(yearly)
    
    # Click the "show" button to load the data
    show = driver.find_element(By.CLASS_NAME, "wrap_btn")
    show.click()

    movieN = []
    # Wait for the page to load and parse it
    page = driver.page_source
    soup = BeautifulSoup(page, "html.parser")
    movies = soup.find("table", "tbl_comm th_sort")
    moviesList = movies.find_all("span", class_="ellip per90")
    for movie in moviesList:
        movie = movie.find("a").text
        movie = movie.strip()
        movie_names.append(movie)
        movieN.append(movie)
        movie_years.append(yearly)
    
    movieL = driver.find_elements(By.ID, "td_audiAcc")
    for movie in movieL:
        movie_viewers.append(movie.text)

    for movie in tqdm(movieN):
        print(movie)
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.LINK_TEXT, movie))).click()

        page = driver.page_source
        soup = BeautifulSoup(page, "html.parser")

        for dt in soup.find_all("dt", text="요약정보"):
            dd = dt.find_next("dd")  # Get the associated <dd> element
            if dd:
                # Extract and clean the genre information
                raw_text = dd.get_text(separator="|").strip()  # Use '|' as separator for clarity
                parts = raw_text.split("|")  # Split by '|'
                genre_parts = [part.strip() for part in parts if "," in part]  # Filter parts with commas (genres)
                for genre in genre_parts:
                    # Further split by commas to separate multiple genres
                    movie_genre.extend([g.strip() for g in genre.split(",")])
                    genres = [
                        '미스터리', '공포(호러)', '액션', '범죄', '판타지', '드라마', 'SF', 
                        '코미디', '뮤지컬', '애니메이션', '스릴러', '어드벤처', '사극', '멜로/로맨스'
                    ]

                    movie_genre = [item for item in movie_genre if item in genres]

        close = driver.find_element(By.CLASS_NAME, "close")
        close.click()

len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

  0%|          | 0/6 [00:00<?, ?it/s]

2019


극한직업


C:\Users\AAA\AppData\Local\Temp\ipykernel_18688\3651094932.py:60: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  for dt in soup.find_all("dt", text="요약정보"):


어벤져스: 엔드게임


...

빅토리


귀멸의 칼날: 인연의 기적, 그리고 합동 강화 훈련으로


무파사: 라이온 킹


100%|██████████| 6/6 [07:23<00:00, 73.95s/it]


(600, 600, 915, 600)

In [47]:
len(movie_names), len(movie_viewers), len(movie_genre), len(movie_years)

(600, 600, 915, 600)

In [54]:
parts

['장편 ',
 ' \n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t일반영화\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 '\n\t                       \t\n\t                       \t\t\t애니메이션\n\t                       \t\n                       \t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t117분 55초\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\n\t\t\t\t\t\t\t\t\n                           \t\t\t전체관람가\n\t\t\t\t\t\t\t\t\n                          \t\n\t\t\t\t\t\t\n\n\t\t\t\t\t\t\n\t\t\t\t\t\t\t',
 '\n\t\t\t\t\t\t\t\n\t                        \t미국\n\t                        \n\t\t\t\t\t\t\n\t\t\t\t\t\t',
 '']

In [51]:
movie_names

['7번방의 선물',
 '설국열차',
 '관상',
 '아이언맨 3',
 '베를린',
 '은밀하게 위대하게',
 '변호인',
 '숨바꼭질',
 '더 테러 라이브',
 '감시자들',
 '월드 워 Z',
 '신세계',
 '박수건달',
 '스파이',
 '그래비티',
 '타워',
 '감기',
 '토르: 다크 월드',
 '레드: 더 레전드',
 '친구 2',
 '어바웃 타임',
 '나우 유 씨 미 : 마술사기단',
 '소원',
 '레미제라블',
 '퍼시픽 림',
 '화이: 괴물을 삼킨 아이',
 '컨저링',
 '맨 오브 스틸',
 '용의자',
 '호빗: 스마우그의 폐허',
 '몽타주',
 '터보',
 '연애의 온도',
 '지.아이.조 2',
 '분노의 질주: 더 맥시멈',
 '공범',
 '집으로 가는 길',
 '전설의 주먹',
 '파파로티',
 '스타트렉 다크니스',
 '라이프 오브 파이',
 '오블리비언',
 '위대한 개츠비',
 '다이하드: 굿 데이 투 다이',
 '런닝맨',
 '미스터 고',
 '결혼전야',
 '깡철이',
 '엘리시움',
 '더 웹툰: 예고살인',
 '명량',
 '겨울왕국',
 '인터스텔라',
 '해적: 바다로 간 산적',
 '수상한 그녀',
 '변호인',
 '국제시장',
 '트랜스포머: 사라진 시대',
 '군도: 민란의 시대',
 '엣지 오브 투모로우',
 '엑스맨: 데이즈 오브 퓨처 패스트',
 '어메이징 스파이더맨 2',
 '타짜-신의 손',
 '혹성탈출: 반격의 서막',
 '캡틴 아메리카: 윈터 솔져',
 '역린',
 '님아, 그 강을 건너지 마오',
 '신의 한 수',
 '끝까지 간다',
 '비긴 어게인',
 '드래곤 길들이기 2',
 '표적',
 '메이즈 러너',
 '호빗: 다섯 군대 전투',
 '나의 사랑 나의 신부',
 '논스톱',
 '인투 더 스톰',
 '노아',
 '남자가 사랑할 때',
 '루시',
 '용의자',
 '기술자들',
 '나를 찾아줘',
 '제보자',
 '피끓는 청춘',
 '두근두근 내 인생',
 '우아한 거짓말',
 '

In [50]:
movie_genre

['코미디',
 '드라마',
 'SF',
 '액션',
 '사극',
 '드라마',
 '액션',
 '어드벤처',
 'SF',
 '액션',
 '코미디',
 '드라마',
 '범죄',
 '범죄',
 '액션',
 '드라마',
 '스릴러',
 'SF',
 '액션',
 '어드벤처',
 '범죄',
 '액션',
 'SF',
 '드라마',
 '드라마',
 '어드벤처',
 '액션',
 '액션',
 '어드벤처',
 '판타지',
 '액션',
 '코미디',
 '범죄',
 '멜로/로맨스',
 '코미디',
 '범죄',
 '액션',
 '스릴러',
 '액션',
 'SF',
 '어드벤처',
 '액션',
 '스릴러',
 '공포(호러)',
 '스릴러',
 '액션',
 '스릴러',
 '어드벤처',
 '드라마',
 '판타지',
 '멜로/로맨스',
 '코미디',
 '액션',
 '어드벤처',
 'SF',
 '스릴러',
 '드라마',
 '드라마',
 '액션',
 '액션',
 'SF',
 '어드벤처',
 '어드벤처',
 '드라마',
 '액션',
 'SF',
 '드라마',
 '멜로/로맨스',
 '액션',
 '범죄',
 '스릴러',
 '액션',
 '코미디',
 '코미디',
 '드라마',
 '멜로/로맨스',
 '코미디',
 '드라마',
 '액션',
 'SF',
 '액션',
 '공포(호러)',
 '스릴러',
 '사극',
 '액션',
 '애니메이션',
 '어드벤처',
 '코미디',
 '뮤지컬',
 '판타지',
 '액션',
 'SF',
 '어드벤처',
 '사극',
 '액션',
 '액션',
 'SF',
 '액션',
 '판타지',
 'SF',
 '어드벤처',
 '액션',
 '판타지',
 '액션',
 '드라마',
 'SF',
 '스릴러',
 '액션',
 '어드벤처',
 'SF',
 '범죄',
 '액션',
 '액션',
 '스릴러',
 '범죄',
 '애니메이션',
 '액션',
 '어드벤처',
 '액션',
 '드라마',
 '액션',
 '미스터리',
 'SF',
 '스릴러',
 '어드벤처',
 '판타지',
 '멜로/로맨스',
 '코

In [49]:
data = {"Name" : movie_names, "Viewers" : movie_viewers, "Year" : movie_years}
df = pd.DataFrame(data)
df.head()

,Name,Viewers,Year
0,7번방의 선물,"12,811,206",2013
1,설국열차,"9,349,993",2013
2,관상,"9,134,238",2013
3,아이언맨 3,"9,001,331",2013
4,베를린,"7,166,290",2013
